In [26]:
import pandas as pd
import numpy as np
import pyodbc
import openpyxl
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from pathlib import Path
from openpyxl import load_workbook, styles, formatting
import sys
import os
import smartsheet
from datetime import datetime, timedelta
import urllib.request
import win32com.client
from numpy import size
from openpyxl.worksheet.page import PageMargins
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles.fonts import Font
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill, Protection

In [27]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
def All_IVY():
    smartsheet_client = smartsheet.Smartsheet("$$$$$$$$")
    smartsheet_client.Sheets.get_sheet_as_csv(6709477773731716, r'C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\IVY')
    # Request Form 신청 리스트 읽어내기 
    df = pd.read_csv(r'C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\IVY\Salesman Account Change Request.csv')
    print(df)
    df_1 = df[df["Requester\'s Email"].notna()]
    print(df_1)
    df = df_1.loc[df_1['Check']==False]
    print(df)
    del df["Requester\'s mail0"]
    print(df)
    if df["Requester\'s Email"].empty==False:
        for i in df.index:
            Account_Num_List = df['Account Number List'][i]
            Account_Num = Account_Num_List.split()
            # GET ACCOUNT NAME, ADDRESS FROM SQL TABLE
            server = '10.1.3.25'
            database = 'KIRA' 
            username = 'kiradba'
            password = 'Kiss!234!'  
            connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
            connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
            engine = create_engine(connection_url)
            conn = engine.raw_connection()
            cursor = conn.cursor()
            print("Connection Established:")
            new = df["New Salesman [A]"][i].split('. ')[1]
            curr = df['Current Salesman [B]'][i].split('. ')[1]
            cursor = conn.cursor()
            val2 = str("'"+new+"'")
            print(val2)
            query1 = f"SELECT distinct salesteam_key from [ivy.mm.dim.sales_master] where salesperson_text={val2}"
            B = cursor.execute(query1)
            NewR = cursor.fetchall()
            NewR = str(NewR).strip("[()]""'"",")
            NewR = str(NewR)[:-3]
            print(NewR)
            val3 = str("'"+curr+"'")
            query2 = f"SELECT distinct salesteam_key from [ivy.mm.dim.sales_master] where salesperson_text = {val3}"
            C = cursor.execute(query2)
            CurrR = cursor.fetchall()
            CurrR = str(CurrR).strip("[()]""'"",")
            CurrR = str(CurrR)[:-3]
            if len(CurrR) >4:
                CurrR = CurrR.replace("', )",'')
                CurrR = CurrR.replace("('",'')
            else:
                CurrR =CurrR
            conn = engine.raw_connection()
            cursor = conn.cursor()
            
            Account_shipto = [];Account_dba = []; Account_add = [];Account_state=[]
            for d in range(0, len(Account_Num)):
                cursor = conn.cursor()
                val = str("'"+Account_Num[d]+"'")
                query_string = f"SELECT shiptoparty from [ivy.mm.dim.shiptoparty] where shiptoparty={val}"
                a = cursor.execute(query_string)
                result = cursor.fetchall()
                result = str(result).strip("[()]""'"",")
                Account_shipto.append(str(result)[:-3])
                cursor = conn.cursor()
                query_string2 = f"SELECT shiptoparty_dba from [ivy.mm.dim.shiptoparty] where shiptoparty ={val}"
                result1 = cursor.execute(query_string2).fetchall()
                result1 = str(result1).strip("[()]""'"",")
                Account_dba.append(str(result1)[:-3])
                query_string3 = f"SELECT address from [ivy.mm.dim.shiptoparty] where shiptoparty ={val}"
                result2 = cursor.execute(query_string3).fetchall()
                result2 = str(result2).strip("[()]""'"",")
                Account_add.append(result2[:-3])
                query_string4 = f"SELECT state_key from [ivy.mm.dim.shiptoparty] where shiptoparty ={val}"
                cursor.execute(query_string4)
                result3 = cursor.fetchall()
                result3 = str(result3).strip("[()]""'"",")
                Account_state.append(result3[:-3])
                d+=1
            data = {'shiptoparty':Account_shipto, 'shiptoparty_dba':Account_dba, 'address':Account_add, 'state_key':Account_state}
            Account_Name = pd.DataFrame(data) 

            Account_number = Account_Name['shiptoparty'].tolist()
            Account_dba = Account_Name['shiptoparty_dba'].tolist()
            Account_Address = Account_Name['address'].tolist()
            State_key = Account_Name['state_key'].tolist()
            # 
            req = df["Requester\'s Email"][i]
            Requester = np.repeat(req, len(Account_Name))
            new= df["New Salesman [A]"][i].split('. ')[1]
            NewMan = np.repeat(new, len(Account_Name))
            curr = df['Current Salesman [B]'][i].split('. ')[1]
            CurrMan = np.repeat(curr, len(Account_Name))
            start = df['Starting Date'][i]
            Start = np.repeat(start, len(Account_Name))
            rea = df['Reason'][i]
            Reason = np.repeat(rea, len(Account_Name))
            CurrR = np.repeat(CurrR, len(Account_Name))
            NewR = np.repeat(NewR, len(Account_Name))
      
            # 
            CurrSalesOffice = CurrR.tolist()
            NewSalesOffice = NewR.tolist()
            Newman = NewMan.tolist()
            Currman = CurrMan.tolist()
            Start = Start.tolist()
            Check = []
            for item in Account_Num:
                if item in Account_number:
                    Check.append("OK")
                else:
                    Check.append("FAIL")
            AA = {'Account_Number':Account_Num, 'System Check':Check}
            df2 = pd.DataFrame(AA, columns = ['Account_Number','System Check'])
            RD = pd.DataFrame({'Requester':Requester, 'New Salesman':Newman, 'Sales Office for New Salesman':NewSalesOffice,'Current Salesman':Currman,'Sales Office for Current Salesman':CurrSalesOffice, 'Start Date':Start, 'Account Number': Account_number, 'Account Name':Account_dba,'Address':Account_Address,'State':State_key })
            print(RD)
            request_num = df['Request Number'][i]
            writer = pd.ExcelWriter(r"C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\IVY"+"\\"+str(request_num)+".xlsx", engine = 'xlsxwriter')
            with writer:
                RD.to_excel(writer, sheet_name = 'Sheet1',index = False, startrow=0,startcol=0)
                df2.to_excel(writer, sheet_name = 'Sheet1', index = False,startcol=12, startrow = 0)
                for column in RD:
                    column_length = max(RD[column].astype(str).map(len).max(), len(column))
                    col_idx = RD.columns.get_loc(column)
                    writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_length)
                for column in df2:
                    column_length = max(df2[column].astype(str).map(len).max(), len(column))
                    col_idx = df2.columns.get_loc(column)
                    writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_length)
                writer.sheets['Sheet1'].set_column(1,1,18)
                writer.sheets['Sheet1'].set_column(12,12,20)
                writer.sheets['Sheet1'].set_column(13,13,18)
                writer.sheets['Sheet1'].set_column(0,0,20)
                writer.sheets['Sheet1'].set_column(7,7,30)
                writer.sheets['Sheet1'].set_column(8,8,30)
            writer.save()
            
        num = list(df.index)
        for i in range(0, len(df.index)):
            smartsheet_client = smartsheet.Smartsheet("%%%%%%%%%%%%%%%")
            sheet = smartsheet_client.Sheets.get_sheet(7653584801163140)
            newRow = smartsheet_client.models.Row()
            newCell = smartsheet_client.models.Cell()
            slice = num[i]
            request_number = df['Request Number'][slice]
            print(request_number)
            newRow.cells.append({'column_id':4669087177041796, 'object_value':str(request_number)})
            newRow.cells.append({'column_id':2416406593070980, 'object_value':str(df["Requester\'s Email"][slice])})
            newRow.cells.append({'column_id': 3948214816139140, 'object_value':str(df["Requester\'s Comment"][slice])})
            import datetime
            today = datetime.date.today().strftime("%m/%d/%Y")
            newRow.cells.append({'column_id':6920886990727044, 'object_value':str(today)})
            smartsheet_client.Sheets.add_rows(7653584801163140, newRow)
            sheet = smartsheet_client.Sheets.get_sheet(7653584801163140)
            for row in sheet.rows:
                thisRow = row.id
            Name = str(str(request_number)+'.xlsx')
            smartsheet_client.Attachments.attach_file_to_row(7653584801163140,thisRow, (Name, open(r"C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\IVY"+"\\"+str(request_number)+".xlsx",'rb'),'application/vnd.ms-excel') )
        now = datetime.datetime.now()
        outlook = win32com.client.Dispatch('Outlook.Application')
        send_mail = outlook.CreateItem(0)
        send_mail.To = '%%^^^$$$em'
        send_mail.Subject = 'SalesAccount Success Email - IVY'
        send_mail.Send()
        print(now)
    else:
        pass


In [29]:
All_IVY()

   Request Number  Requester's mail0     Requester's Email New Salesman [A]  \
0               1                NaN                   NaN              NaN   
1               2                NaN                   NaN              NaN   
2               3                NaN                   NaN              NaN   
3               4                NaN                   NaN              NaN   
4               5                NaN                   NaN              NaN   
5               6                NaN                   NaN              NaN   
6              A7                NaN    edwardh@ivyent.com         BRIAN KO   
7              A8                NaN    edwardh@ivyent.com    HAE SANG PYUN   
8           IVY11                NaN  miles.kim@ivyent.com       SAMUEL KIM   
9           IVY13                NaN      yelee@ivyent.com   146. ALICE LEE   
10          IVY14                NaN    edwardh@ivyent.com    167. BRIAN KO   
11          IVY15                NaN    edwardh@ivye

In [30]:
def All_RED():
    from datetime import date
    from datetime import datetime
    smartsheet_client = smartsheet.Smartsheet("%%%%%%%%%%%%%%")
    smartsheet_client.Sheets.get_sheet_as_csv(5486522041427844, r'C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\RED')
    # Request Form 신청 리스트 읽어내기 
    df = pd.read_csv(r'C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\RED\Salesman Account Change Request [RED].csv')
    print(df)
    df_1 = df[df["Requester\'s Email"].notna()]
    print(df_1)
    df = df_1.loc[df_1['Check']==False]
    print(df)
    del df["Primary Column"]
    print(df)
    newdf = df.notnull()
    if df["Requester\'s Email"].empty==False:
        for i in df.index:
            Account_Num_List = df['Account Number List'][i]
            
            Account_Num = Account_Num_List.split()
            
            print(Account_Num)
            # GET ACCOUNT NAME, ADDRESS FROM SQL TABLE
            server = '@@@@@@@@@'
            database = '!!!!!!' 
            username = '%%%%%%%%%'
            password = '^^^^^^^^^'  
            connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
            connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
            engine = create_engine(connection_url)
            print("Connection Established:")
            
            conn = engine.raw_connection()
            
            new = df["New Salesman [A]"][i].split('. ')[1]
            curr = df['Current Salesman [B]'][i].split('. ')[1]
            cursor = conn.cursor()
            val2 = str("'"+new+"'")
            print(val2)
            query1 = f"SELECT distinct salesteam_key from [ivy.mm.dim.sales_master] where salesperson_text={val2}"
            B = cursor.execute(query1)
            NewR = cursor.fetchall()
            NewR = str(NewR).strip("[()]""'"",")
            NewR = str(NewR)[:-3]
            print(NewR)
            val3 = str("'"+curr+"'")
            query2 = f"SELECT distinct salesteam_key from [ivy.mm.dim.sales_master] where salesperson_text = {val3}"
            C = cursor.execute(query2)
            CurrR = cursor.fetchall()
            
            CurrR = str(CurrR).strip("[()]""'"",")
            CurrR = CurrR[:-3]
            if len(CurrR) >4:
                CurrR = CurrR.replace("', )",'')
                CurrR = CurrR.replace("('",'')
            else:
                CurrR =CurrR
            conn = engine.raw_connection()
            cursor = conn.cursor()
            Account_shipto = [];Account_dba = []; Account_add = [];Account_state=[]
            for item in Account_Num:
                val = str("'"+item+"'")
                query_string = f"SELECT shiptoparty from [ivy.mm.dim.shiptoparty] where shiptoparty={val}"
                a = cursor.execute(query_string)
                result = cursor.fetchall()
                result = str(result).strip("[()]""'"",")
                Account_shipto.append(result[:-3])
                cursor = conn.cursor()
                query_string2 = f"SELECT shiptoparty_dba from [ivy.mm.dim.shiptoparty] where shiptoparty ={val}"
                result1 = cursor.execute(query_string2).fetchall()
                result1 = str(result1).strip("[()]""'"",")
                Account_dba.append(str(result1)[:-3])
                query_string3 = f"SELECT address from [ivy.mm.dim.shiptoparty] where shiptoparty ={val}"
                result2 = cursor.execute(query_string3).fetchall()
                result2 = str(result2).strip("[()]""'"",")
                Account_add.append(result2[:-3])
                query_string4 = f"SELECT state_key from [ivy.mm.dim.shiptoparty] where shiptoparty ={val}"
                cursor.execute(query_string4)
                result3 = cursor.fetchall()
                result3 = str(result3).strip("[()]""'"",")
                Account_state.append(result3[:-3])
                
            data = {'shiptoparty':Account_shipto, 'shiptoparty_dba':Account_dba, 'address':Account_add, 'state_key':Account_state}
            Account_Name = pd.DataFrame(data) 

            Account_number = Account_Name['shiptoparty'].tolist()
            Account_dba = Account_Name['shiptoparty_dba'].tolist()
            Account_Address = Account_Name['address'].tolist()
            State_key = Account_Name['state_key'].tolist()
            print(Account_number)
            print(Account_dba)
            print(Account_Address)
            # 
            req = df["Requester\'s Email"][i]
            Requester = np.repeat(req, len(Account_Name))
            new = df["New Salesman [A]"][i].split('. ')[1]
            NewMan = np.repeat(new, len(Account_Name))
            curr = df['Current Salesman [B]'][i].split('. ')[1]
            CurrMan = np.repeat(curr, len(Account_Name))
            start = df['Starting Date'][i]
            Start = np.repeat(start, len(Account_Name))
            rea = df['Reason'][i]
            Reason = np.repeat(rea, len(Account_Name))
            CurrR = np.repeat(CurrR, len(Account_Name))
            NewR = np.repeat(NewR, len(Account_Name))
      
            # 
            CurrSalesOffice = CurrR.tolist()
            NewSalesOffice = NewR.tolist()
            Newman = NewMan.tolist()
            Currman = CurrMan.tolist()
            Start = Start.tolist()
            Check = []
            for item in Account_Num:
                if item in Account_number:
                    Check.append("OK")
                else:
                    Check.append("FAIL")
            AA={'Original Input Number':Account_Num, 'System Check':Check}
            df2 = pd.DataFrame(AA, columns = ['Original Input Number','System Check'])
            RD = pd.DataFrame({'Requester':Requester, 'New Salesman':Newman, 'Sales Office for New Salesman':NewSalesOffice,'Current Salesman':Currman,'Sales Office for Current Salesman':CurrSalesOffice, 'Start Date':Start, 'Account Number': Account_number, 'Account Name':Account_dba,'Address':Account_Address,'State':State_key })
            print(RD)
            request_num = df['Request Number'][i]
            writer = pd.ExcelWriter(r"C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\RED"+"\\"+str(request_num)+".xlsx", engine = 'xlsxwriter')
            with writer:
                RD.to_excel(writer, sheet_name = 'Sheet1',index = False, startrow = 0, startcol = 0)
                df2.to_excel(writer,sheet_name = 'Sheet1',index = False,startrow = 0, startcol=12)
                for column in RD:
                    column_length = max(RD[column].astype(str).map(len).max(), len(column))
                    col_idx = RD.columns.get_loc(column)
                    writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_length)
                for column in df2:
                    column_length = max(df2[column].astype(str).map(len).max(), len(column))
                    col_idx2 = df2.columns.get_loc(column)
                    writer.sheets['Sheet1'].set_column(col_idx2, col_idx2, column_length)
                writer.sheets['Sheet1'].set_column(1,1,18)
                writer.sheets['Sheet1'].set_column(12,12,20)
                writer.sheets['Sheet1'].set_column(13,13,18)
                writer.sheets['Sheet1'].set_column(0,0,20)
                writer.sheets['Sheet1'].set_column(7,7,30)
                writer.sheets['Sheet1'].set_column(8,8,30)
            writer.save()
            
        num = list(df.index)
        for i in range(0,len(df.index)):
            smartsheet_client = smartsheet.Smartsheet("%%%%%%%%%%")
            sheet = smartsheet_client.Sheets.get_sheet(6621597340592004)
            newRow = smartsheet_client.models.Row()
            newCell = smartsheet_client.models.Cell()
            slice = num[i]
            request_number = df['Request Number'][slice]
            print(request_number)
            newRow.cells.append({'column_id':3635223067420548, 'object_value':str(request_number)})
            newRow.cells.append({'column_id':8138822694791044, 'object_value':str(df["Requester\'s Email"][slice])})
            newRow.cells.append({'column_id': 9005538807179140, 'object_value':str(df["Requester\'s Comment"][slice])})
            today = date.today().strftime("%m/%d/%Y")
            
            newRow.cells.append({'column_id':820473300313988, 'object_value':str(today)})
            smartsheet_client.Sheets.add_rows(6621597340592004, newRow)
            sheet = smartsheet_client.Sheets.get_sheet(6621597340592004)
            for row in sheet.rows:
                thisRow = row.id
            Name = str(str(request_number)+'.xlsx')
            smartsheet_client.Attachments.attach_file_to_row(6621597340592004,thisRow, (Name, open(r"C:\Users\KISS Admin\OneDrive - Kiss Products Inc\Desktop\salesman_account_management\Salesman Account Change\RED"+"\\"+str(request_number)+".xlsx",'rb'),'application/vnd.ms-excel') )
        
        
        outlook = win32com.client.Dispatch('Outlook.Application')
        send_mail = outlook.CreateItem(0)
        send_mail.To = '^^^^^^^^^^'
        send_mail.Subject = 'SalesAccount Success Email - RED'
        send_mail.Send()
        
    else:
        pass

In [31]:
All_RED()

    Primary Column Request Number                 Requester's Email  \
0              NaN              5  Miles Kim <miles.kim@ivyent.com>   
1              NaN              6  Miles Kim <miles.kim@ivyent.com>   
2              NaN              7  Miles Kim <miles.kim@ivyent.com>   
3              NaN          RED13            noah.lee@redbeauty.com   
4              NaN          RED14            noah.lee@redbeauty.com   
5              NaN          RED15                edwardh@ivyent.com   
6              NaN          RED17            noah.lee@redbeauty.com   
7              NaN          RED18            noah.lee@redbeauty.com   
8              NaN          RED19                edwardh@ivyent.com   
9              NaN          RED20          brandon1.choi@ivyent.com   
10             NaN          RED22                edwardh@ivyent.com   
11             NaN          RED23            noah.lee@redbeauty.com   
12             NaN          RED24            noah.lee@redbeauty.com   

     

In [32]:
import win32com.client
import schedule
import datetime
import time


schedule.every(20).minutes.do(All_IVY)
schedule.every(20).minutes.do(All_RED)
while True:
    schedule.run_pending()
    
    time.sleep(1)



   Request Number  Requester's mail0     Requester's Email New Salesman [A]  \
0               1                NaN                   NaN              NaN   
1               2                NaN                   NaN              NaN   
2               3                NaN                   NaN              NaN   
3               4                NaN                   NaN              NaN   
4               5                NaN                   NaN              NaN   
5               6                NaN                   NaN              NaN   
6              A7                NaN    edwardh@ivyent.com         BRIAN KO   
7              A8                NaN    edwardh@ivyent.com    HAE SANG PYUN   
8           IVY11                NaN  miles.kim@ivyent.com       SAMUEL KIM   
9           IVY13                NaN      yelee@ivyent.com   146. ALICE LEE   
10          IVY14                NaN    edwardh@ivyent.com    167. BRIAN KO   
11          IVY15                NaN    edwardh@ivye

C:\Users\KISS Admin\AppData\Local\Temp\ipykernel_7728\4263976176.py:133: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\KISS Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


IVY16
2023-03-31 09:42:28.660569
    Primary Column Request Number                 Requester's Email  \
0              NaN              5  Miles Kim <miles.kim@ivyent.com>   
1              NaN              6  Miles Kim <miles.kim@ivyent.com>   
2              NaN              7  Miles Kim <miles.kim@ivyent.com>   
3              NaN          RED13            noah.lee@redbeauty.com   
4              NaN          RED14            noah.lee@redbeauty.com   
5              NaN          RED15                edwardh@ivyent.com   
6              NaN          RED17            noah.lee@redbeauty.com   
7              NaN          RED18            noah.lee@redbeauty.com   
8              NaN          RED19                edwardh@ivyent.com   
9              NaN          RED20          brandon1.choi@ivyent.com   
10             NaN          RED22                edwardh@ivyent.com   
11             NaN          RED23            noah.lee@redbeauty.com   
12             NaN          RED24           

C:\Users\KISS Admin\AppData\Local\Temp\ipykernel_7728\4263976176.py:133: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\KISS Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


IVY16
2023-03-31 09:42:42.216546
    Primary Column Request Number                 Requester's Email  \
0              NaN              5  Miles Kim <miles.kim@ivyent.com>   
1              NaN              6  Miles Kim <miles.kim@ivyent.com>   
2              NaN              7  Miles Kim <miles.kim@ivyent.com>   
3              NaN          RED13            noah.lee@redbeauty.com   
4              NaN          RED14            noah.lee@redbeauty.com   
5              NaN          RED15                edwardh@ivyent.com   
6              NaN          RED17            noah.lee@redbeauty.com   
7              NaN          RED18            noah.lee@redbeauty.com   
8              NaN          RED19                edwardh@ivyent.com   
9              NaN          RED20          brandon1.choi@ivyent.com   
10             NaN          RED22                edwardh@ivyent.com   
11             NaN          RED23            noah.lee@redbeauty.com   
12             NaN          RED24           

C:\Users\KISS Admin\AppData\Local\Temp\ipykernel_7728\4263976176.py:133: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\KISS Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


IVY16
2023-03-31 09:42:45.802173
    Primary Column Request Number                 Requester's Email  \
0              NaN              5  Miles Kim <miles.kim@ivyent.com>   
1              NaN              6  Miles Kim <miles.kim@ivyent.com>   
2              NaN              7  Miles Kim <miles.kim@ivyent.com>   
3              NaN          RED13            noah.lee@redbeauty.com   
4              NaN          RED14            noah.lee@redbeauty.com   
5              NaN          RED15                edwardh@ivyent.com   
6              NaN          RED17            noah.lee@redbeauty.com   
7              NaN          RED18            noah.lee@redbeauty.com   
8              NaN          RED19                edwardh@ivyent.com   
9              NaN          RED20          brandon1.choi@ivyent.com   
10             NaN          RED22                edwardh@ivyent.com   
11             NaN          RED23            noah.lee@redbeauty.com   
12             NaN          RED24           

C:\Users\KISS Admin\AppData\Local\Temp\ipykernel_7728\4263976176.py:133: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\KISS Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


IVY16
2023-03-31 09:42:47.701282
    Primary Column Request Number                 Requester's Email  \
0              NaN              5  Miles Kim <miles.kim@ivyent.com>   
1              NaN              6  Miles Kim <miles.kim@ivyent.com>   
2              NaN              7  Miles Kim <miles.kim@ivyent.com>   
3              NaN          RED13            noah.lee@redbeauty.com   
4              NaN          RED14            noah.lee@redbeauty.com   
5              NaN          RED15                edwardh@ivyent.com   
6              NaN          RED17            noah.lee@redbeauty.com   
7              NaN          RED18            noah.lee@redbeauty.com   
8              NaN          RED19                edwardh@ivyent.com   
9              NaN          RED20          brandon1.choi@ivyent.com   
10             NaN          RED22                edwardh@ivyent.com   
11             NaN          RED23            noah.lee@redbeauty.com   
12             NaN          RED24           

KeyboardInterrupt: 